# Task 1: Third-Order Letter Approximation Model  

In [26]:
# Imports.
# Selecting random items from lists.
import random
# Efficient data structures.
import collections
import json

Step 1: A method that will read in the text from one of the books

In [27]:
#method to read in a book
def readBook(book):
    with open(book, 'r', encoding='utf-8') as file:#open the file
        text = file.read()#store the read in file in a variable
    return text#return the text

In [28]:
#Clean the text by putting it to lower case and only keeping ascii chars
def cleanBook(text):
    #sentences that are at the start and end of the actual content
    startOfBook = "*** START OF THE PROJECT GUTENBERG EBOOK"
    endOfBook = "*** END OF THE PROJECT GUTENBERG EBOOK"
    #Make the text uppercase, and cut out the preambe and postambe of the project
    #gutenburg books I picked
    #https://www.freecodecamp.org/news/how-to-split-a-string-in-python/
    english = text.upper();
    english = english.split(startOfBook, 1)[-1]
    english = english.split(endOfBook, 1)[0]
    # The characters to keep.
    keepTheseCharacters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ .'
    # Remove unwanted characters.
    cleanedText = ''.join(c for c in english if c in keepTheseCharacters)
    
    #get rif of new lines note to self
    return cleanedText


Building the trigram model

In [29]:
#method to build the trigram model by counting how many times 3 char sequences show up
def makeTrigramModel(book):
    #create a default int dictionary
    trigramModel = collections.defaultdict(int)

    for i in range(len(book) - 2):
        #this is getting the 3 char sequence
        trigram = book[i:i+3]
        #increment the count
        trigramModel[trigram] += 1
    #retun the built trigram model
    return trigramModel

Combine all 5 works to make bigger data for the model

In [ ]:
works = [readBook(f"books/voyaging.txt") for i in range(1, 6)]
cleanedWorks = cleanBook(works)
combinedTexts = ' '.join(cleanedWorks)

FileNotFoundError: [Errno 2] No such file or directory: 'books/book1.txt'

In [ ]:
trigramModel = makeTrigramModel(combinedTexts)
print(trigramModel)

defaultdict(<class 'int'>, {'The': 502, 'he ': 5826, 'e P': 10, ' Pr': 10, 'Pro': 5, 'roj': 5, 'oje': 5, 'jec': 50, 'ect': 435, 'ct ': 145, 't G': 47, ' Gu': 362, 'Gut': 2, 'ute': 59, 'ten': 314, 'enb': 3, 'nbe': 4, 'ber': 87, 'erg': 21, 'rg ': 2, 'g e': 37, ' eB': 2, 'eBo': 3, 'Boo': 3, 'ook': 380, 'ok ': 153, 'k o': 131, ' of': 2059, 'of ': 1937, 'f A': 32, ' Al': 1123, 'Ald': 1345, 'ldy': 1345, 'dyt': 1345, 'yth': 1433, "th'": 150, "h's": 149, "'s ": 604, 's i': 378, ' in': 1731, 'inh': 35, 'nhe': 33, 'her': 3662, 'eri': 188, 'rit': 167, 'ita': 97, 'tan': 398, 'anc': 315, 'nce': 599, 'ce\n': 42, 'e\n ': 2, '\n  ': 91, '   ': 553, '  \n': 1, ' \nT': 1, '\nTh': 132, 'Thi': 33, 'his': 942, 'is ': 1617, 's e': 164, ' eb': 5, 'ebo': 4, 'boo': 26, 'k i': 72, ' is': 822, 's f': 305, ' fo': 1075, 'for': 1170, 'or ': 947, 'r t': 671, ' th': 6384, 'the': 5060, 'e u': 115, ' us': 142, 'use': 317, 'se ': 550, 'e o': 784, 'f a': 215, ' an': 2444, 'any': 291, 'nyo': 1, 'yon': 13, 'one': 548, 'ne 

TASK 2

In [ ]:
#get the trigram counts for each matching trigram
def getCounts(model, trigrams):
    counts = []
    for trigram in trigrams:
        counts.append(model[trigram])
    return counts

In [ ]:
#Get the matching trigrams that start with the 2 characters
def getMatchingTrigrams(model, twoChars):
    matchingTrigrams = []

    for trigram in model:
        if trigram.startswith(twoChars):
            matchingTrigrams.append(trigram)

    if matchingTrigrams:
        counts = getCounts(model, matchingTrigrams)
        return matchingTrigrams, counts
    else:
        return None, None

In [ ]:
#pick the next char of a trigram based on the random counts choice
def pickNextCharacter(trigrams, counts):
    #return only the third character
    chosenTrigram = random.choices(trigrams, counts, k=1)[0]
    return chosenTrigram[-1]

Generate the Text by using the Trigram model created earlier

In [ ]:
def generateText(model, startText="TH", length=10000):
    #start the generated text by storing the initial string given in the task
    generatedText = startText
    #keep going till the text is 10000
    while len(generatedText) < length:
        #take out the last 2 chars from the current text
        lastTwoChars = generatedText[-2:]
        #call the method that gets all the trigrams that start with the current 2 chars and their counts
        trigrams, counts = getMatchingTrigrams(model, lastTwoChars)
        #if there are matching trigrams then pick the next char based on the probablity of the counts
        if trigrams:
            nextCharacter = pickNextCharacter(trigrams, counts)
            generatedText += nextCharacter
        #if there are no mathcing trigrams stop generating the text
        else:
            break

    return generatedText

In [ ]:
genText = generateText(trigramModel, startText="TH", length=10000)
print(genText)

THREWELIZED

"Of he wasmis ithance," en thin thells ming tawyestred he poreas unmed to suarrey wellood ton Cro yousnothou nat to pearesips, Cappropeas she this Lor as!" andoings?"

But sper
frot youse wis Lor grobblettes glined saiddining."

Somed Kitty to me ung go mily. She war an dut lighty to con is ever And
ther come onal, Ald beembase. Staid nandham."

Mr. He way: Ande cou mang, all bell, Guy?" shentme, I dockle hut ind Ald ank oully a lad a paraing a dep to ling. I herroake troult im to maked a hat wook
youlty dines.

"But wought. Hills wind in doeme fic-hun Wynnoth's whaptan Poethand thaver Guy, bed to mys mind cam sh. I se!" had Glack onderhad senwor
to re sy at onagive girle ne fire orne wither spid pin And Guy candhatureat tharce to bet, ing toods highters sh ingloned to could of and
man to whist ught on worris ver heyonowit of cand
of Ald resell bat's streladied the welf sinuou any.

"I wooke dolou weld
rit she wou to anxien Lorry thary arda's plied thingave me ye he fell t

Task 3 

In [ ]:
#get all the words from words.txt and store them in a set to get easily
def getAllTheWords(filePath):
    with open(filePath, 'r') as file:
        words = set(line.strip().upper() for line in file)
    return words

In [ ]:
#take the words out 
def takeOutWords(text):
    #split by spaces
    return text.split()

In [ ]:
#count the valid words and calculate the percentage
def calculatePercentage(text, listOfWords):
    words = takeOutWords(text)
    wordCount = 0
    totalWords = len(words)

    for word in words:
        if word in listOfWords:
            wordCount += 1
    
    if totalWords == 0:
        return 0
    
    return (wordCount/totalWords)*100

In [ ]:
englishWordList = getAllTheWords('words.txt')
percentage = calculatePercentage(genText, englishWordList)
print(percentage)

0.055524708495280406


## Task 4 Export the Model as JSON

In [ ]:
with open('trigrams.json', 'w') as file:
    json.dump(trigramModel, file)